In [1]:
import torch

from torch import nn, optim



from torch.autograd import Variable

from torch.utils.data import DataLoader

from torchvision import transforms

from torchvision import datasets

In [2]:
batch_size = 32

learning_rate = 1e-2

num_epoches = 50

In [3]:
# 下载训练集 MNIST 手写数字训练集

train_dataset = datasets.MNIST(

    root='./data', train=True, transform=transforms.ToTensor(), download=True)



test_dataset = datasets.MNIST(

    root='./data', train=False, transform=transforms.ToTensor())



train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# 定义简单的前馈神经网络

class Neuralnetwork(nn.Module):

    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):

        super(Neuralnetwork, self).__init__()

        self.layer1 = nn.Linear(in_dim, n_hidden_1)

        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)

        self.layer3 = nn.Linear(n_hidden_2, out_dim)



    def forward(self, x):

        x = self.layer1(x)

        x = self.layer2(x)

        x = self.layer3(x)

        return x

In [5]:
model = Neuralnetwork(28 * 28, 300, 100, 10)

if torch.cuda.is_available():

    model = model.cuda()



criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)



In [8]:
for epoch in range(num_epoches):

    print('epoch {}'.format(epoch + 1))

    print('*' * 10)

    running_loss = 0.0

    running_acc = 0.0

    for i, data in enumerate(train_loader, 1):

        img, label = data

        img = img.view(img.size(0), -1)

        if torch.cuda.is_available():

            img = Variable(img).cuda()

            label = Variable(label).cuda()

        else:

            img = Variable(img)

            label = Variable(label)

        # 向前传播

        out = model(img)

        loss = criterion(out, label)

        running_loss += loss.item() * label.size(0)

        _, pred = torch.max(out, 1)

        num_correct = (pred == label).sum()

        running_acc += num_correct.item()

        # 向后传播

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()



        if i % 300 == 0:

            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(

                epoch + 1, num_epoches, running_loss / (batch_size * i),

                running_acc / (batch_size * i)))

    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(

        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(

            train_dataset))))

    model.eval()

    eval_loss = 0.

    eval_acc = 0.

    for data in test_loader:

        img, label = data

        img = img.view(img.size(0), -1)

        if torch.cuda.is_available():

            img = Variable(img, volatile=True).cuda()

            label = Variable(label, volatile=True).cuda()

        else:

            img = Variable(img, volatile=True)

            label = Variable(label, volatile=True)

        out = model(img)

        loss = criterion(out, label)

        eval_loss += loss.item() * label.size(0)

        _, pred = torch.max(out, 1)

        num_correct = (pred == label).sum()

        eval_acc += num_correct.item()

    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(

        test_dataset)), eval_acc / (len(test_dataset))))

    print()



epoch 1
**********
[1/50] Loss: 0.392620, Acc: 0.888958
[1/50] Loss: 0.394535, Acc: 0.887708
[1/50] Loss: 0.383305, Acc: 0.890764
[1/50] Loss: 0.376706, Acc: 0.892474
[1/50] Loss: 0.371869, Acc: 0.893813
[1/50] Loss: 0.365297, Acc: 0.896111
Finish 1 epoch, Loss: 0.363977, Acc: 0.896533


I:\Anaconda\lib\site-packages\ipykernel_launcher.py:87: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
I:\Anaconda\lib\site-packages\ipykernel_launcher.py:89: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test Loss: 0.325940, Acc: 0.905800

epoch 2
**********
[2/50] Loss: 0.333252, Acc: 0.906146
[2/50] Loss: 0.334839, Acc: 0.905365
[2/50] Loss: 0.329931, Acc: 0.906181
[2/50] Loss: 0.329324, Acc: 0.906042
[2/50] Loss: 0.329295, Acc: 0.906646
[2/50] Loss: 0.325630, Acc: 0.907153
Finish 2 epoch, Loss: 0.325682, Acc: 0.907350
Test Loss: 0.304571, Acc: 0.913700

epoch 3
**********
[3/50] Loss: 0.316526, Acc: 0.906667
[3/50] Loss: 0.308466, Acc: 0.911979
[3/50] Loss: 0.308252, Acc: 0.912431
[3/50] Loss: 0.308161, Acc: 0.912656
[3/50] Loss: 0.309345, Acc: 0.912167
[3/50] Loss: 0.308709, Acc: 0.912604
Finish 3 epoch, Loss: 0.307819, Acc: 0.912867
Test Loss: 0.294732, Acc: 0.917100

epoch 4
**********
[4/50] Loss: 0.295222, Acc: 0.916875
[4/50] Loss: 0.292770, Acc: 0.916562
[4/50] Loss: 0.298080, Acc: 0.916181
[4/50] Loss: 0.296866, Acc: 0.916250
[4/50] Loss: 0.297499, Acc: 0.916292
[4/50] Loss: 0.298635, Acc: 0.915260
Finish 4 epoch, Loss: 0.297450, Acc: 0.915483
Test Loss: 0.286208, Acc: 0.918

Finish 26 epoch, Loss: 0.253029, Acc: 0.929950
Test Loss: 0.271120, Acc: 0.924500

epoch 27
**********
[27/50] Loss: 0.254045, Acc: 0.930833
[27/50] Loss: 0.252717, Acc: 0.928281
[27/50] Loss: 0.254951, Acc: 0.928299
[27/50] Loss: 0.253755, Acc: 0.929427
[27/50] Loss: 0.253360, Acc: 0.929271
[27/50] Loss: 0.253739, Acc: 0.929913
Finish 27 epoch, Loss: 0.253230, Acc: 0.930133
Test Loss: 0.270851, Acc: 0.923400

epoch 28
**********
[28/50] Loss: 0.249377, Acc: 0.931250
[28/50] Loss: 0.248863, Acc: 0.932396
[28/50] Loss: 0.251658, Acc: 0.932326
[28/50] Loss: 0.252603, Acc: 0.931172
[28/50] Loss: 0.252195, Acc: 0.930750
[28/50] Loss: 0.252976, Acc: 0.930556
Finish 28 epoch, Loss: 0.252475, Acc: 0.930400
Test Loss: 0.268320, Acc: 0.926200

epoch 29
**********
[29/50] Loss: 0.242651, Acc: 0.933333
[29/50] Loss: 0.252287, Acc: 0.930000
[29/50] Loss: 0.253601, Acc: 0.930035
[29/50] Loss: 0.251171, Acc: 0.930625
[29/50] Loss: 0.253319, Acc: 0.930438
[29/50] Loss: 0.251585, Acc: 0.930816
Finish 

In [9]:
# 保存模型

torch.save(model.state_dict(), './neural_network.pth')